<a href="https://colab.research.google.com/github/Solrak97/tf-vgg16-flowerdata/blob/main/VGG_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de aprendizaje utilizando VGG16

In [1]:
%%capture
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! rm -rf flowers
! kaggle datasets download alxmamaev/flowers-recognition
! unzip flowers-recognition.zip
! rm -r flowers-recognition.zip
! rm -rf flowers/sample_data/

In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
import pickle

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Carga del dataset

In [3]:
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                      transforms.ColorJitter(brightness=1, contrast=1, saturation=1),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])


transform = transforms.Compose([transforms.Resize((224,224)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

dataset = ImageFolder("flowers")
train, val = torch.utils.data.random_split(dataset, [4317 - 863, 863], generator=torch.Generator().manual_seed(42))

train.dataset.transform = transform_train
val.dataset.transform = transform

training_loader = torch.utils.data.DataLoader(train, batch_size=200, shuffle=True)
validation_loader = torch.utils.data.DataLoader(val, batch_size = 200, shuffle=False)

In [4]:
def im_convert(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

## Modificación del modelo base VGG16

In [5]:
classes = ('margaritas', 'rosas', 'dientes de león', 'tulipanes', 'girasoles')

In [6]:
# Creación del modelo VGG16
model = models.vgg16(pretrained=True)

# Congelamiento del modelo base
for param in model.features.parameters():
  param.requires_grad = False

n_inputs = model.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(classes))

# Reemplzo de la capa de salida
model.classifier[6] = last_layer
#model.classifier.append(nn.Softmax())
model = model.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Entrenamiento del modelo

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [8]:
epochs = 10
running_loss_history = []
running_corrects_history = []
val_running_loss_history = []
val_running_corrects_history = []

for e in range(epochs):
  
  running_loss = 0.0
  running_corrects = 0.0
  val_running_loss = 0.0
  val_running_corrects = 0.0
  
  for inputs, labels in training_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item()
    running_corrects += torch.sum(preds == labels.data)

  else:
    with torch.no_grad():
      for val_inputs, val_labels in validation_loader:
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)

        _, val_preds = torch.max(val_outputs, 1)
        val_running_loss += val_loss.item()
        val_running_corrects += torch.sum(val_preds == val_labels.data)
      
    epoch_loss = running_loss/len(training_loader.dataset)
    epoch_acc = running_corrects.float()/ len(training_loader.dataset)
    running_loss_history.append(epoch_loss)
    running_corrects_history.append(epoch_acc)
    
    val_epoch_loss = val_running_loss/len(validation_loader.dataset)
    val_epoch_acc = val_running_corrects.float()/ len(validation_loader.dataset)
    val_running_loss_history.append(val_epoch_loss)
    val_running_corrects_history.append(val_epoch_acc)
    print('epoch :', (e+1))
    print('training loss: {:.4f}, acc {:.4f} '.format(epoch_loss, epoch_acc.item()))
    print('validation loss: {:.4f}, validation acc {:.4f} '.format(val_epoch_loss, val_epoch_acc.item()))

epoch : 1
training loss: 0.0040, acc 0.7131 
validation loss: 0.0025, validation acc 0.8366 
epoch : 2
training loss: 0.0014, acc 0.9062 
validation loss: 0.0021, validation acc 0.8563 
epoch : 3
training loss: 0.0006, acc 0.9647 
validation loss: 0.0022, validation acc 0.8702 
epoch : 4
training loss: 0.0003, acc 0.9858 
validation loss: 0.0023, validation acc 0.8737 
epoch : 5
training loss: 0.0002, acc 0.9933 
validation loss: 0.0023, validation acc 0.8749 
epoch : 6
training loss: 0.0001, acc 0.9977 
validation loss: 0.0026, validation acc 0.8598 
epoch : 7
training loss: 0.0001, acc 0.9986 
validation loss: 0.0024, validation acc 0.8714 
epoch : 8
training loss: 0.0000, acc 0.9991 
validation loss: 0.0029, validation acc 0.8691 
epoch : 9
training loss: 0.0000, acc 0.9988 
validation loss: 0.0026, validation acc 0.8737 
epoch : 10
training loss: 0.0000, acc 0.9994 
validation loss: 0.0028, validation acc 0.8656 


## Guardar modelo

In [9]:
def save_checkpoint(model, path):
    # Extract the final classifier and the state dictionary
    
    state = {}

    # Check to see if model was parallelized
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        state['classifier'] = model.module.classifier
        state['state_dict'] = model.module.state_dict()

    else:
       state['classifier'] = model.classifier
       state['state_dict'] = model.state_dict()

    # Save the data to the path
    torch.save(state, path)

## Cargar modelo

In [10]:
def load_checkpoint(path, device):

    # Load in checkpoint
    state = torch.load(path, map_location=torch.device('cpu'))

    # Non trainable model
    model = models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False


    # Load in the state dict
    model.classifier = state['classifier']
    model.load_state_dict(state['state_dict'])

    return model.to(device)

In [11]:
save_checkpoint(model, 'classifier.trch')
load_checkpoint('classifier.trch', device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1